In [43]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거
dir_path = './data/'

In [67]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [45]:
# cid = pd.read_excel('aladin_Category_CID_20210927.xls')
# cid = cid[['CID', '몰', '카테고리명',  '1Depth', '2Depth']]
# cid.columns = ['CID', 'mall', 'category_name',  '1Depth', '2Depth']
# cid

In [46]:
import os
import sys
import requests
import json

#키 정의
key = 'ttbsmk9805101528002'

## 1.1. Best seller(Book)

In [57]:
book_list1 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList&Cover=Big"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        #book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['cover'] = item['cover']
        #book_dict['salesPoint'] = item['salesPoint']
        #book_dict['CID'] = item['categoryId']
        #book_dict['customer_review_rank'] = item['customerReviewRank']
        #book_dict['pricesales'] = item['priceSales']
        #book_dict['pricestandard'] = item['priceStandard']
        #book_dict['pubdate'] = item['pubDate']
        #book_dict['publisher'] = item['publisher']

        book_list1.append(book_dict)
best_book = pd.DataFrame(book_list1)
best_df = best_book.drop_duplicates()

In [ ]:
# best_df = pd.merge(best_book, cid, how='left', left_on='CID', right_on='CID')
# best_df

## 1.2. Best seller(e-book)

In [58]:
book_list2 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=Bestseller&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList&Cover=Big"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        #book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['cover'] = item['cover']
        #book_dict['salesPoint'] = item['salesPoint']
        #book_dict['CID'] = item['categoryId']
        #book_dict['customer_review_rank'] = item['customerReviewRank']
        #book_dict['pricesales'] = item['priceSales']
        #book_dict['pricestandard'] = item['priceStandard']
        #book_dict['pubdate'] = item['pubDate']
        #book_dict['publisher'] = item['publisher']

        book_list2.append(book_dict)
best_e_book = pd.DataFrame(book_list2)
best_e_df = best_e_book.drop_duplicates()

In [59]:
# best_e_df = pd.merge(best_e_book, cid, how='left', left_on='CID', right_on='CID')
# best_e_df

## 1.3. Best seller(Book + eBook)

In [62]:
best = pd.concat((best_df, best_e_df))
best = best.drop_duplicates()
best.index = [i for i in range(len(best.index))]
best['isbn'].value_counts()

0                8
                 5
9788962630619    2
9788934931836    1
9791141108380    1
9791192548791    1
9791190075466    1
9791166385070    1
9788997228355    1
9791135975424    1
9791133675968    1
9791141108403    1
9791169850285    1
9791168967687    1
9788925525839    1
9791138138604    1
9791169732697    1
9791155514924    1
9791140432400    1
9791167741004    1
9791139117356    1
9791132540991    1
9791139908022    1
9791169850216    1
9788901269634    1
9791127446031    1
9791132266228    1
9791196700591    1
9788927765103    1
9788925500003    1
9791197746208    1
9791170693376    1
9791168473690    1
9791137339033    1
9791192166940    1
9791141108328    1
9791168129542    1
9791132878186    1
9791170621546    1
9791139809282    1
9791137336025    1
9791169385176    1
9791138138598    1
9791140674817    1
9791170620075    1
9791138418263    1
9791130699684    1
9791190762830    1
9791135974977    1
9791190538527    1
9791126591329    1
9791169603256    1
978202304192

## 2.1. blog best(Book)

In [63]:
book_list3 = []

for i in range(1, 5):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=BlogBest&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList&Cover=Big"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        #book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['cover'] = item['cover']
        #book_dict['salesPoint'] = item['salesPoint']
        #book_dict['CID'] = item['categoryId']
        #book_dict['customer_review_rank'] = item['customerReviewRank']
        #book_dict['pricesales'] = item['priceSales']
        #book_dict['pricestandard'] = item['priceStandard']
        #book_dict['pubdate'] = item['pubDate']
        #book_dict['publisher'] = item['publisher']

        book_list3.append(book_dict)
blog_book = pd.DataFrame(book_list3)
blog_df = blog_book.drop_duplicates()

In [64]:
# blog_df = pd.merge(blog_book, cid, how='left', left_on='CID', right_on='CID')
# blog_df

## 3.1. Best + Blog

In [66]:
best_blog = pd.concat((best, blog_df))
best_blog = best_blog.drop_duplicates()
best_blog.index = [i for i in range(len(best_blog.index))]
best_blog['isbn'].value_counts()
len(best_blog)

2081

In [68]:
best_blog['isbn'].value_counts()

0                8
                 5
9788962630619    2
9791168473690    1
9788956057897    1
                ..
9791130344812    1
9791191925128    1
9791127868611    1
9788954692250    1
9791192674605    1
Name: isbn, Length: 2069, dtype: int64

## 4.1 New Special(Book)

In [69]:
book_list4 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=Book" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList&Cover=Big"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        #book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['cover'] = item['cover']
        #book_dict['salesPoint'] = item['salesPoint']
        #book_dict['CID'] = item['categoryId']
        #book_dict['customer_review_rank'] = item['customerReviewRank']
        #book_dict['pricesales'] = item['priceSales']
        #book_dict['pricestandard'] = item['priceStandard']
        #book_dict['pubdate'] = item['pubDate']
        #book_dict['publisher'] = item['publisher']

        book_list4.append(book_dict)
new_book = pd.DataFrame(book_list4)
new_book = new_book.drop_duplicates()

In [70]:
# new_book = pd.merge(new_book, cid, how='left', left_on='CID', right_on='CID')
# new_book.index = [i for i in range(1, len(new_book.index) + 1)]
# new_book

## 4.2 New Special(e-book)

In [71]:
book_list5 = []

for i in range(1, 21):
    url = f"http://www.aladin.co.kr/ttb/api/ItemList.aspx?ttbkey={key}&QueryType=ItemNewSpecial&SearchTarget=eBook" \
            f"&MaxResults=50&start={i}&output=js&Version=20131101&OptResult=usedList&Cover=Big"
    response = requests.get(url)
    response_json = json.loads(response.text)['item']
    
    for item in response_json:
        book_dict = {}
        book_dict['isbn'] = item['isbn13']
        #book_dict['itemid'] = item['itemId']
        book_dict['title'] = item['title']
        book_dict['description'] = item['description']
        book_dict['author'] = item['author']
        book_dict['cover'] = item['cover']
        #book_dict['salesPoint'] = item['salesPoint']
        #book_dict['CID'] = item['categoryId']
        #book_dict['customer_review_rank'] = item['customerReviewRank']
        #book_dict['pricesales'] = item['priceSales']
        #book_dict['pricestandard'] = item['priceStandard']
        #book_dict['pubdate'] = item['pubDate']
        #book_dict['publisher'] = item['publisher']

        book_list5.append(book_dict)
new_ebook = pd.DataFrame(book_list5)
new_ebook = new_ebook.drop_duplicates()

In [72]:
# new_ebook = pd.merge(new_ebook, cid, how='left', left_on='CID', right_on='CID')
# new_ebook.index = [i for i in range(1, len(new_ebook.index) + 1)]
# new_ebook

## 4.3 New Special(Book + ebook)

In [73]:
new_df = pd.concat((new_book, new_ebook))
new_df = new_df.drop_duplicates()
new_df.index = [i for i in range(len(new_df.index))]
new_df

,isbn,title,description,author,cover
0,9791130699318,우리들의 MBTI 4 : 학습 유형 - 나의 성격을 이해하고 더 멋진 내가 되는,,"조수연 (지은이), 소윤 (그림)",https://image.aladin.co.kr/product/31674/58/co...
1,9788986836899,행복의 건축 (리커버),"일상의 철학가 알랭 드 보통이 바라본, 건축에 관한 특별한 시선. 우리의 가장 강렬...","알랭 드 보통 (지은이), 정영목 (옮긴이)",https://image.aladin.co.kr/product/31674/31/co...
2,9788932405193,에밀리 디킨슨 시 선집,&lt;에밀리 디킨슨 시 선집&gt;이 을유세계문학전집 126번째 작품으로 출간되었...,"에밀리 디킨슨 (지은이), 조애리 (옮긴이)",https://image.aladin.co.kr/product/31673/40/co...
3,9788954692991,여학교의 별 3,,"와야마 야마 (지은이), 현승희 (옮긴이)",https://image.aladin.co.kr/product/31673/28/co...
4,9791160079241,52주의 숄 - 사계절 내내 즐기는 아름다운 손뜨개 스카프,사계절 내내 즐길 수 있는 숄과 스카프 52가지를 소개한다. 여름철 실내에서 가볍게...,"레인 (지은이), 조진경 (옮긴이)",https://image.aladin.co.kr/product/31673/14/co...
...,...,...,...,...,...
1176,9788969525352,진짜 수학 공부법 - 류승재 특급 비법,"공부, 그중에서도 학생들이 가장 어려워하는 수학을 잘하는 공부법을 다룬다. 수학 공...",류승재 (지은이),https://image.aladin.co.kr/product/31132/69/co...
1177,9791197918155,자미 - 내 이름의 새로운 철자,《자미》는 우리 시대 페미니스트들의 가장 중요한 원천이자 목소리인 오드리 로드의 자...,"오드리 로드 (지은이), 송섬별 (옮긴이)",https://image.aladin.co.kr/product/31062/48/co...
1178,9788964627303,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),https://image.aladin.co.kr/product/31033/95/co...
1179,9791169794107,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",https://image.aladin.co.kr/product/30961/16/co...


## 5.1 Best + Blog + New Special

In [74]:
len(best_blog), len(new_df)

(2081, 1181)

In [75]:
best_blog_new = pd.concat((best_blog, new_df))
best_blog_new = best_blog_new.drop_duplicates()
best_blog_new.index = [i for i in range(len(best_blog_new.index))]
best_blog_new

,isbn,title,description,author,cover
0,9791168473690,세이노의 가르침,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",세이노 (지은이),https://image.aladin.co.kr/product/30929/51/co...
1,9791169631402,화산귀환 3,대 화산파 13대 제자. 천하삼대검수 매화검존 청명. 천하를 혼란에 빠뜨린 고금제일...,"STUDIO LICO (지은이), 비가 (원작)",https://image.aladin.co.kr/product/31632/20/co...
2,9788954692526,각각의 계절,유려하고도 엄정한 문장의 아름다움을 일깨우며 한국문학이 신뢰하는 이름이 된 작가 권...,권여선 (지은이),https://image.aladin.co.kr/product/31599/26/co...
3,9791167740984,"도둑맞은 집중력 - 집중력 위기의 시대, 삶의 주도권을 되찾는 법",우리는 집중하지 못하고 산만해지는 것이 흔히 스마트폰과 같은 디지털 기기에 대해 자...,"요한 하리 (지은이), 김하현 (옮긴이)",https://image.aladin.co.kr/product/31559/97/co...
4,9791190299770,모든 삶은 흐른다,2022년 프랑스 최고의 철학과 교수로 꼽힌 로랑스 드빌레르의 인문에세이로 출간 후...,"로랑스 드빌레르 (지은이), 이주영 (옮긴이)",https://image.aladin.co.kr/product/31381/47/co...
...,...,...,...,...,...
2924,9788969525352,진짜 수학 공부법 - 류승재 특급 비법,"공부, 그중에서도 학생들이 가장 어려워하는 수학을 잘하는 공부법을 다룬다. 수학 공...",류승재 (지은이),https://image.aladin.co.kr/product/31132/69/co...
2925,9791197918155,자미 - 내 이름의 새로운 철자,《자미》는 우리 시대 페미니스트들의 가장 중요한 원천이자 목소리인 오드리 로드의 자...,"오드리 로드 (지은이), 송섬별 (옮긴이)",https://image.aladin.co.kr/product/31062/48/co...
2926,9788964627303,반도체 삼국지,"반도체공학자이자 첨단산업 분야의 전략가 권석준 교수가 한국, 일본, 그리고 중국 반...",권석준 (지은이),https://image.aladin.co.kr/product/31033/95/co...
2927,9791169794107,수호자들,"《수호자들》은 《타임 투 킬》, 《펠리컨 브리프》 등 수많은 베스트셀러를 배출하고 ...","존 그리샴 (지은이), 남명성 (옮긴이)",https://image.aladin.co.kr/product/30961/16/co...


In [76]:
best_blog_new['isbn'].value_counts()

                 14
0                 8
9791190299886     2
9788962630619     2
9788965706625     1
                 ..
9788957365793     1
9791156757344     1
9791162203620     1
9791169279178     1
9791164840601     1
Name: isbn, Length: 2907, dtype: int64

In [77]:
best_blog_new = best_blog_new.drop(best_blog_new[(best_blog_new['isbn'] == '0') | (best_blog_new['isbn'] == '')].index) \
                .reset_index(drop=True)
print(best_blog_new['isbn'].value_counts())
best_blog_new.drop_duplicates(['isbn'], inplace=True, ignore_index=True)
best_blog_new['isbn'].value_counts()

9791190299886    2
9788962630619    2
9791168473690    1
9791192862385    1
9788934983170    1
                ..
9788957365793    1
9791156757344    1
9791162203620    1
9791169279178    1
9791164840601    1
Name: isbn, Length: 2905, dtype: int64


9791168473690    1
9791168125537    1
9791141105068    1
9788934983170    1
9788997213733    1
                ..
9788957365793    1
9791156757344    1
9791162203620    1
9791169279178    1
9791164840601    1
Name: isbn, Length: 2905, dtype: int64

In [30]:
best_blog_new = best_blog_new.drop(best_blog_new[best_blog_new['description'] == ''].index) \
                .reset_index(drop=True)
len(best_blog_new)

2303

In [80]:
best_blog_new.to_excel(dir_path + "aladin_20230515_final.xlsx", index = False, header = True)

## Data Update

In [79]:
df1 = pd.read_excel(dir_path + "aladin_20230515_final.xlsx", engine='openpyxl')

In [33]:
df2 = pd.read_excel(dir_path + "aladin_20230502.xlsx", engine='openpyxl')

In [34]:
df = pd.concat([df1, df2], ignore_index=True)
df.drop_duplicates(['isbn'], inplace=True, ignore_index=True)
len(df)

2383

In [35]:
df['isbn'].value_counts()

9791168473690    1
9791167700605    1
9788925576671    1
9788964946008    1
9788965965657    1
                ..
9791138343435    1
9791198189202    1
9791168126251    1
9788954655972    1
9791158748593    1
Name: isbn, Length: 2383, dtype: int64

In [36]:
df.to_excel(dir_path + "aladin_20230515_final.xlsx", index = False, header = True)